In [3]:
pip install textblob

In [4]:
pip install autocorrect

Note: you may need to restart the kernel to use updated packages.


In [5]:
import pandas as pd
import re 
from textblob import TextBlob
from autocorrect import Speller 
from textblob import Word 
from nltk.corpus import stopwords 
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

In [6]:
data = pd.read_csv('amazon_alexa.tsv',sep='\t')
data.head(5)

,rating,date,variation,verified_reviews,feedback
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1
4,5,31-Jul-18,Charcoal Fabric,Music,1


In [7]:
len(data)

3150

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3150 entries, 0 to 3149
Data columns (total 5 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   rating            3150 non-null   int64 
 1   date              3150 non-null   object
 2   variation         3150 non-null   object
 3   verified_reviews  3150 non-null   object
 4   feedback          3150 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 123.2+ KB


In [9]:
data.verified_reviews.isnull().sum()

0

In [10]:
data.feedback.isnull().sum()

0

In [11]:
data.feedback.value_counts()

1    2893
0     257
Name: feedback, dtype: int64

In [12]:
reviews=data.verified_reviews
reviews

0                                           Love my Echo!
1                                               Loved it!
2       Sometimes while playing a game, you can answer...
3       I have had a lot of fun with this thing. My 4 ...
4                                                   Music
                              ...                        
3145    Perfect for kids, adults and everyone in betwe...
3146    Listening to music, searching locations, check...
3147    I do love these things, i have them running my...
3148    Only complaint I have is that the sound qualit...
3149                                                 Good
Name: verified_reviews, Length: 3150, dtype: object

In [13]:
new_list=[]
for i in reviews:
    new_string = re.sub(r"[^a-zA-Z0-9]"," ",i)
    new_string = new_string.lower() 
    new_list.append(new_string)

df = pd.DataFrame(new_list)
df

,0
0,love my echo
1,loved it
2,sometimes while playing a game you can answer...
3,i have had a lot of fun with this thing my 4 ...
4,music
...,...
3145,perfect for kids adults and everyone in betwe...
3146,listening to music searching locations check...
3147,i do love these things i have them running my...
3148,only complaint i have is that the sound qualit...


In [14]:
lst = []
for i in new_list:
    i = re . sub ( '[\s]+' , ' ' , i )
    i  = re . sub ( '[\n]+' ,' ',i)
    lst.append(i)
    
df= pd.DataFrame(lst)
df

,0
0,love my echo
1,loved it
2,sometimes while playing a game you can answer ...
3,i have had a lot of fun with this thing my 4 y...
4,music
...,...
3145,perfect for kids adults and everyone in between
3146,listening to music searching locations checkin...
3147,i do love these things i have them running my ...
3148,only complaint i have is that the sound qualit...


In [16]:
new_lst = []

for i in lst:
    sent = " ".join([Word(word).lemmatize() for word in i.split()])
    new_lst.append(sent)

reviews = pd.DataFrame(new_lst)
reviews

,0
0,love my echo
1,loved it
2,sometimes while playing a game you can answer ...
3,i have had a lot of fun with this thing my 4 y...
4,music
...,...
3145,perfect for kid adult and everyone in between
3146,listening to music searching location checking...
3147,i do love these thing i have them running my e...
3148,only complaint i have is that the sound qualit...


In [17]:
stop_words = set(stopwords.words('english'))

In [18]:
temp = []

for i in new_lst:
    s=" ".join([ w for w in i.split() if w not in stop_words])
    temp.append(s)
    
df = pd.DataFrame(temp)
df

,0
0,love echo
1,loved
2,sometimes playing game answer question correct...
3,lot fun thing 4 yr old learns dinosaur control...
4,music
...,...
3145,perfect kid adult everyone
3146,listening music searching location checking ti...
3147,love thing running entire home tv light thermo...
3148,complaint sound quality great mostly use comma...


In [20]:
newData=pd.concat([data,df],axis=1)
newData.head()

,rating,date,variation,verified_reviews,feedback,0
0,5,31-Jul-18,Charcoal Fabric,Love my Echo!,1,love echo
1,5,31-Jul-18,Charcoal Fabric,Loved it!,1,loved
2,4,31-Jul-18,Walnut Finish,"Sometimes while playing a game, you can answer...",1,sometimes playing game answer question correct...
3,5,31-Jul-18,Charcoal Fabric,I have had a lot of fun with this thing. My 4 ...,1,lot fun thing 4 yr old learns dinosaur control...
4,5,31-Jul-18,Charcoal Fabric,Music,1,music


In [21]:
newData=newData.drop(['rating','date','variation','verified_reviews'],axis=1)

newData.rename(columns={0:'review'},inplace=True)
newData

,feedback,review
0,1,love echo
1,1,loved
2,1,sometimes playing game answer question correct...
3,1,lot fun thing 4 yr old learns dinosaur control...
4,1,music
...,...,...
3145,1,perfect kid adult everyone
3146,1,listening music searching location checking ti...
3147,1,love thing running entire home tv light thermo...
3148,1,complaint sound quality great mostly use comma...


In [22]:
newData = newData.sample(frac=1)
newData.head()

,feedback,review
787,1,good sound work well
1526,1,great sound screen nice actually see time triv...
1417,1,love item underestimated item another 2 cart w...
1404,1,pleasedsimple figure
314,1,bought daughter turned 9 love especially like ...


In [23]:
from sklearn.model_selection import train_test_split 
X_train,X_test,Y_train,Y_test = train_test_split(newData['review'],newData['feedback'],test_size=0.25,random_state=30)
print('X_train : ',X_train.shape)
print('Y_train : ',Y_train.shape)
print('X_test : ',X_test.shape)
print('Y_test : ',Y_test.shape)

X_train :  (2362,)
Y_train :  (2362,)
X_test :  (788,)
Y_test :  (788,)


In [24]:
X_train

1081             love clock tell set alarm fussing button
2794                           perfect kid adult everyone
680                     ha need life plan time date music
1947    built hub lack feature geo fencing phillips hu...
1094                                                     
                              ...                        
86                                            clear music
352                          work great different new one
2048                                                     
1720                                 absolutely love show
1975                                                     
Name: review, Length: 2362, dtype: object

In [25]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [26]:
vectorizer = TfidfVectorizer()
tf_x_train = vectorizer.fit_transform(X_train)
tf_x_test = vectorizer.transform(X_test)

In [27]:
tf_x_train.shape

(2362, 3121)

In [28]:
from sklearn.svm import LinearSVC
clf = LinearSVC(random_state=0)

In [30]:
clf.fit(tf_x_train,Y_train)

LinearSVC(random_state=0)

In [31]:
y_test_pred = clf.predict(tf_x_test)

In [32]:
from sklearn.metrics import accuracy_score 
accuracy_score(Y_test,y_test_pred)

0.9428934010152284

In [33]:
df=pd.DataFrame(Y_test)
df["Predicted_val"] = pd.DataFrame(y_test_pred)

In [34]:
Y_test.value_counts()

1    731
0     57
Name: feedback, dtype: int64

In [35]:
df['Predicted_val'].value_counts()

1.0    186
0.0      5
Name: Predicted_val, dtype: int64

In [37]:
def analyze_sentiment(review):
    analysis=TextBlob(review)
    if analysis.sentiment.polarity>0:
        return 'Positive'
    elif analysis.sentiment.polarity==0:
        return 'Neutral'
    else:
        return 'Negative'

In [39]:
newData['Sentiment'] = newData['review'].apply(lambda x : analyze_sentiment(x))
newData[['review','Sentiment']].head(5)

,review,Sentiment
787,good sound work well,Positive
1526,great sound screen nice actually see time triv...,Positive
1417,love item underestimated item another 2 cart w...,Positive
1404,pleasedsimple figure,Neutral
314,bought daughter turned 9 love especially like ...,Positive
